In [1]:
# import matplotlib
# matplotlib.use("qtagg")
%matplotlib qt
import os
root = os.path.realpath(os.getcwd()+ os.sep+"..")
import sys
sys.path.append(root)
from modules.WiFi import WiFi, Modes, make_command
import pathfinder, pathfinder_visualizer
import importlib
import matplotlib.pyplot as plt
import matplotlib.animation as anim
import time
import re, json
from datetime import datetime

In [ ]:
def show_visual():
    def on_hover_point(point_polar):
        pathfinder.set_goal(point_polar) if not point_polar is None else None
    fig, update_func = pathfinder_visualizer.run_visualizer(pathfinder, on_hover_point)
    anime = anim.FuncAnimation(fig, update_func, 1, interval=50, blit=True)
    plt.show()
    return anime
# show_visual()

from ipywidgets import Button, VBox, ToggleButton, IntSlider, HBox, HTML, Layout, Output

start_pathfinder = Button(
    description="Start Pathfinder",
    layout = Layout(width='auto', height='80px', margin='10px'), 
    style=dict(font_size="60px"),
    button_style="primary",
    disabled = pathfinder._service.is_running())
stop_pathfinder = Button(
    description="Stop Pathfinder", 
    layout= start_pathfinder.layout, 
    style=start_pathfinder.style,
    button_style="warning",
    disabled = not pathfinder._service.is_running())
run_rover = ToggleButton(
    value=False, 
    description="Run Rover", 
    layout= Layout(flex='1', height='80px', margin='10px'), 
    style=dict(font_size="60px"),
    button_style="success",
    disabled = not pathfinder._service.is_running())
visual_button = Button(
    description="PLT", 
    layout = Layout(height='auto', margin='10px'))
speed_slide = IntSlider(
    value=0, min=0, max=100, 
    orientation="vertical", 
    layout=Layout(height="auto"))
# text_display = HTML(
#     value="Some HTML text", 
#     # layout= Layout(flex='1', margin="10px", border="solid", padding="10px"))
#     layout= )
text_output = Output(layout=Layout(flex='1', max_height="500px", min_height="100px"))
poll_rover = Button(
    description="Poll Rover",
    layout = Layout(flex='1')
)
clear_output = Button(
    description="Clear Output"
)

buttons = VBox([
    start_pathfinder,
    HBox([run_rover,], layout=Layout(width='auto', padding='0px', margin='0px')),
    stop_pathfinder,
    ],
    layout = Layout(flex='1')
)
controls = HBox([
    speed_slide, buttons, speed_slide
])
logging = HBox([
    HBox([text_output], layout=Layout(border='solid', flex='1', padding='10px')), VBox([poll_rover, clear_output])
    ],
    layout = Layout(margin='10px')
)
gui = VBox([
    controls,
    logging
])
# buttons

called start_path_finder_service
Starting Pathfinder Service
Called start_worldview_service
Starting Worldview Service
Called start_lidar_service
Starting Lidar Service

started Lidar Scanning Thread


Too many measurments in the input buffer: 501/500. Clearing buffer...
Too many measurments in the input buffer: 538/500. Clearing buffer...
Too many measurments in the input buffer: 512/500. Clearing buffer...
Too many measurments in the input buffer: 583/500. Clearing buffer...
Too many measurments in the input buffer: 501/500. Clearing buffer...
Too many measurments in the input buffer: 590/500. Clearing buffer...
Too many measurments in the input buffer: 508/500. Clearing buffer...
Too many measurments in the input buffer: 585/500. Clearing buffer...
Too many measurments in the input buffer: 522/500. Clearing buffer...
Too many measurments in the input buffer: 577/500. Clearing buffer...
Too many measurments in the input buffer: 576/500. Clearing buffer...
Too many measurments in the input buffer: 538/500. Clearing buffer...
Too many measurments in the input buffer: 527/500. Clearing buffer...
Too many measurments in the input buffer: 571/500. Clearing buffer...
Too many measurments

Sending command : {'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'S', 'CMD': [0, -26]}


Too many measurments in the input buffer: 546/500. Clearing buffer...


Data sent successfully


...

Too many measurments in the input buffer: 575/500. Clearing buffer...
Too many measurments in the input buffer: 549/500. Clearing buffer...
Too many measurments in the input buffer: 502/500. Clearing buffer...
Too many measurments in the input buffer: 521/500. Clearing buffer...
Too many measurments in the input buffer: 532/500. Clearing buffer...
Too many measurments in the input buffer: 520/500. Clearing buffer...
Too many measurments in the input buffer: 509/500. Clearing buffer...
Too many measurments in the input buffer: 524/500. Clearing buffer...
Too many measurments in the input buffer: 509/500. Clearing buffer...
Too many measurments in the input buffer: 555/500. Clearing buffer...
Too many measurments in the input buffer: 522/500. Clearing buffer...
Too many measurments in the input buffer: 511/500. Clearing buffer...
Too many measurments in the input buffer: 556/500. Clearing buffer...
Too many measurments in the input buffer: 511/500. Clearing buffer...
Too many measurments

called stop_path_finder_service

Called stop_worldview_service
Called stop_lidar_service
waiting for obstacle thread to join
waiting for lidar thread to join
Stopping Lidar Service
Starting Worldview Service
Stopping Pathfinder Service
called stop_path_finder_service
called start_path_finder_service
Starting Pathfinder Service
Called start_worldview_service
Starting Worldview Service
Called start_lidar_service
Starting Lidar Service
called start_path_finder_service
Starting Pathfinder Service
Called start_worldview_service
Starting Worldview Service
Called start_lidar_service


Exception in thread Pathfinder service:
Traceback (most recent call last):
  File "d:\Projects\urc-intelligent-systems-2023\env\Lib\threading.py", line 1045, in _bootstrap_inner
Exception in thread Lidar Service Thread:
Traceback (most recent call last):
  File "d:\Projects\urc-intelligent-systems-2023\env\Lib\site-packages\rplidar.py", line 126, in connect
    self.run()
  File "d:\Projects\urc-intelligent-systems-2023\env\Lib\threading.py", line 982, in run
    self._serial_port = serial.Serial(
                        ^^^^^^^^^^^^^^
  File "d:\Projects\urc-intelligent-systems-2023\env\Lib\site-packages\serial\serialwin32.py", line 33, in __init__
    self._target(*self._args, **self._kwargs)
  File "d:\Projects\urc-intelligent-systems-2023\unified_frameworks\pathfinder.py", line 184, in run_pathfinder
    exploration_step(obstacles, get_near_points)
  File "d:\Projects\urc-intelligent-systems-2023\unified_frameworks\pathfinder.py", line 46, in inner
    res = func(*args, **kwargs)
 

Starting Lidar Service


    self.open()
  File "d:\Projects\urc-intelligent-systems-2023\env\Lib\site-packages\serial\serialwin32.py", line 64, in open
    raise SerialException("could not open port {!r}: {!r}".format(self.portstr, ctypes.WinError()))
serial.serialutil.SerialException: could not open port 'COM10': PermissionError(13, 'Access is denied.', None, 5)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "d:\Projects\urc-intelligent-systems-2023\unified_frameworks\sensor_array\lidar.py", line 37, in run_lidar
    lidar = RPLidar(PORT_NAME)
            ^^^^^^^^^^^^^^^^^^
  File "d:\Projects\urc-intelligent-systems-2023\env\Lib\site-packages\rplidar.py", line 117, in __init__
    self.connect()
  File "d:\Projects\urc-intelligent-systems-2023\env\Lib\site-packages\rplidar.py", line 131, in connect
    raise RPLidarException('Failed to connect to the sensor '
rplidar.RPLidarException: Failed to connect to the sensor due to: could not open port

In [14]:
from math import pi
from unified_utils import Service

rover = WiFi("http://192.168.0.211:5000")
def captain_act():
    path = pathfinder.get_path()
    if len(path) < 2:
        rover.send_command(make_command(speed_percent=0))
        # speed_slide.value=0
        radians=pi/2
    else:
        nxt = path[1]
        radians = (nxt[0]%(2*pi))-pi/2
    radians*=-1
    degrees = int(radians/(2*pi)*360)
    mode = Modes.DRIVE if abs(degrees) < 12 else Modes.SPIN
    speed = speed_slide.value
    if mode == Modes.SPIN:
        speed *= (-1 if degrees < 0 else 1)
    # print(mode, speed)
    command = make_command(mode, speed_percent=speed, angle_degrees=degrees)
    print(f"Sending command : {command}")
    rover.send_command(command)
def captain_stop():
    rover.send_command(make_command(speed_percent=0))
command_freq = 3
def start_captain(is_captain_running):
    while is_captain_running():
        captain_act()
        # print("running captain")
        time.sleep(1/command_freq)
    # print("stopping captain")
    captain_stop()
    

Too many measurments in the input buffer: 519/500. Clearing buffer...
Too many measurments in the input buffer: 520/500. Clearing buffer...
Too many measurments in the input buffer: 535/500. Clearing buffer...


In [15]:
def update_button_disables(pathfinding):
    start_pathfinder.disabled = pathfinding
    run_rover.disabled = not pathfinding
    stop_pathfinder.disabled = not pathfinding

def on_start_pathfinding(b):
    importlib.reload(pathfinder)
    pathfinder.start_pathfinder_service()
    # show_visual()
    update_button_disables(True)
start_pathfinder.on_click(on_start_pathfinding)
def on_stop_pathfinding(b):
    pathfinder.stop_pathfinder_service()
    update_button_disables(False)
stop_pathfinder.on_click(on_stop_pathfinding)

# def on_visual_toggle(change):
#     show = change['new']
#     print(show)
#     if show:
#         show_visual()
#     else:
#         plt.close('all')
# visual_button.observe(on_visual_toggle, 'value')
def on_visual(b):
    show_visual()
visual_button.on_click(on_visual)

captain_service = Service(start_captain, "Captain Service")
def rover_order_changed(change):
    global captain_service
    run = change['new']
    if run:
        captain_service.start_service()
        run_rover.button_style='danger'
        run_rover.description='Stop Rover'
    else:
        captain_service.stop_service()
        captain_service = Service(start_captain, "Captain Service")
        run_rover.button_style='success'
        run_rover.description='Run Rover'
run_rover.observe(rover_order_changed, 'value')

clear_output.on_click(lambda i: text_output.clear_output())
history = []
def on_poll_rover(b):
    stat = rover.get_status()
    with text_output:
        print()
        print(f"at: {datetime.now().strftime('%H:%M:%S')}")
        print(json.dumps({k:stat[k] for k in stat if re.match("^current.*", k)},indent=2))
poll_rover.on_click(on_poll_rover)

Too many measurments in the input buffer: 503/500. Clearing buffer...
Too many measurments in the input buffer: 514/500. Clearing buffer...
Too many measurments in the input buffer: 501/500. Clearing buffer...
Too many measurments in the input buffer: 508/500. Clearing buffer...
Too many measurments in the input buffer: 558/500. Clearing buffer...
Too many measurments in the input buffer: 521/500. Clearing buffer...
Too many measurments in the input buffer: 565/500. Clearing buffer...
Too many measurments in the input buffer: 535/500. Clearing buffer...
Too many measurments in the input buffer: 569/500. Clearing buffer...
Too many measurments in the input buffer: 512/500. Clearing buffer...


In [16]:
# show_visual()

Too many measurments in the input buffer: 515/500. Clearing buffer...
Too many measurments in the input buffer: 540/500. Clearing buffer...
Too many measurments in the input buffer: 516/500. Clearing buffer...
Too many measurments in the input buffer: 567/500. Clearing buffer...
Too many measurments in the input buffer: 527/500. Clearing buffer...
Too many measurments in the input buffer: 517/500. Clearing buffer...
Too many measurments in the input buffer: 532/500. Clearing buffer...
Too many measurments in the input buffer: 538/500. Clearing buffer...
Too many measurments in the input buffer: 518/500. Clearing buffer...
Too many measurments in the input buffer: 538/500. Clearing buffer...
Too many measurments in the input buffer: 565/500. Clearing buffer...


In [6]:
gui

In [12]:
rover.send_command(make_command(Modes.SPIN,speed_percent=0))

Data sent successfully


Too many measurments in the input buffer: 502/500. Clearing buffer...


# Archive 1

In [ ]:
from ipywidgets import Button, HBox, ToggleButton, TwoByTwoLayout, Layout, IntSlider
# 3 buttons. Start service, stop service, show visualization
make_layout = lambda: Layout(width="auto", height="auto")
start_button = Button(description="Start Pathfinder", disabled=pathfinder._service.is_running(), button_style='primary', layout=make_layout(), style=dict(font_size="60px"))
stop_button = Button(description="Stop Pathfinder", disabled=not pathfinder._service.is_running(), button_style='warning', layout=make_layout(), style=dict(font_size="60px"))
show_button = Button(description="Show Visualization", button_style='warning', layout=make_layout())


pathfinder_controls = TwoByTwoLayout(
    top_left=start_button,
    bottom_left=stop_button,
    layout = Layout(height="300px")
    )
pathfinder_controls

In [ ]:
start_button

In [ ]:
rideon_button = ToggleButton(description="Start Pathfinder", disabled=pathfinder._service.is_running(), button_style='primary', layout=make_layout(), style=dict(font_size="60px"))
stop_button = Button(description="Stop Pathfinder", disabled=not pathfinder._service.is_running(), button_style='warning', layout=make_layout(), style=dict(font_size="60px"))
speed_slide = IntSlider(value=0, min=0, max=100, layout=make_layout())

In [ ]:
def start_pathfinder(*args):
    if pathfinder._service.is_running():
        print("Service is already running", file=sys.stderr)
        return
    pathfinder.start_pathfinder_service()
    stop_button.disabled=False
    start_button.disabled=True
start_button.on_click(start_pathfinder)
def stop_pathfinder(*args):
    pathfinder.stop_pathfinder_service()
    importlib.reload(pathfinder)
    stop_button.disabled=True
    start_button.disabled=False
stop_button.on_click(stop_pathfinder)
# def show_visual(*args):
#     fig, update_func = pathfinder_visualizer.run_visualizer(pathfinder)
#     anime = anim.FuncAnimation(fig, update_func, 1, interval=50, blit=True)
#     plt.show()
#     return anime
# show_button.on_click(show_visual)

In [ ]:
def show_visual():
    def on_hover_point(point_polar):
        pathfinder.set_goal(point_polar) if not point_polar is None else None
    fig, update_func = pathfinder_visualizer.run_visualizer(pathfinder, on_hover_point)
    anime = anim.FuncAnimation(fig, update_func, 1, interval=50, blit=True)
    plt.show()
    return anime

In [ ]:
gui

In [ ]:
from ipywidgets import IntSlider
speed = 10
s = IntSlider(value=speed, min=0, max=25)

In [ ]:
s.value

In [ ]:
s

In [ ]:
from math import pi
import time
rover = WiFi("http://192.168.0.211:5000")
speed = 10
def captain_act():
    path = pathfinder.get_path()
    if len(path) < 2:
        return
    nxt = path[1]
    radians = (nxt[0]%(2*pi))-pi/2
    radians*=-1
    degrees = int(radians/(2*pi)*360)
    mode = Modes.DRIVE if abs(degrees) < 12 else Modes.SPIN
    speed = s.value
    if mode == Modes.SPIN:
        speed *= (-1 if degrees < 0 else 1)
    # print(mode, speed)
    rover.write_data(make_command(mode, speed_percent=speed, angle_degrees=degrees))


In [ ]:
from threading import Thread

In [ ]:
rover.read_data()

In [ ]:
rover.write_data(make_command(Modes.SPIN))

In [ ]:
stop = False

In [ ]:
def f():
    global stop
    print("Staring auto")
    for i in (list(range(100))):
        if stop: break
        captain_act()
        time.sleep(1)
        rover.write_data(make_command(speed_percent=0))
    print("Done auto")
t = Thread(target=f)
t.start()

In [ ]:
rover.write_data(make_command(Modes.DRIVE,speed_percent=0))

In [ ]:
rover.read_data()